### The goal is to get the outline, plan of attack for the implemetation of the paper and test some improvements.
    - pick a sector
    - scrape the web to get the top 300 / 500 / 800 stocks and their trade symbols from yfinance.
    - make a dataframe of the returns for the stocks that you have downloaded.
    - picking the pairs can happen in the permutation of the following:
        - (GARCH or not) clustering of the time series data
        - Double stage correlation and cointegration
        - vanilla cointegration
    - do PCA on the sector indices and construct the eigen portfolio form the eigenvalues.
    - model the parameters of the residual as orienstein uhlenbeck process using the sde package

### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

### The sector I am working with here as an experimentation is pharma

In [2]:
url ='https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=50'

Getting the url's of the pages which contains the data for n stocks from pharma

In [3]:
l=np.arange(0,226,25)
for i in range(len(l)):
    url = 'https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=' + str(l[i])
    print(url)

https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=0
https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=25
https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=50
https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=75
https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=100
https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=125
https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&

In [4]:
list_webpages = []
for i in range(len(l)):
    url = 'https://finance.yahoo.com/screener/unsaved/bc105bad-a0a2-4e16-bd3e-884c0b2771ad?count=25&dependentField=sector&dependentValues=Healthcare&offset=' + str(l[i])
    list_webpages.append(url)

In [1]:
symbol = pd.Series([1])
for i in range(len(list_webpages)):
    df_list = pd.read_html(list_webpages[i])
    symbol = pd.concat([symbol, df_list[0]["Symbol"]], ignore_index = True)

NameError: name 'pd' is not defined

In [ ]:
symbol = pd.DataFrame(symbol)

In [ ]:
symbol = symbol.drop(0)

In [ ]:
symbol = symbol.rename(columns={0: "Symbol"})


In [ ]:
symbol.shape

### Get the Adj close price for all the symbols listed in the dataframe

In [ ]:
dataframe = yf.download(['DIVISLAB.NS', 'DRREDDY.BO'], '2018-01-01', '2019-01-01')

In [ ]:
dataframe['Adj Close']

In [ ]:
symbol["Symbol"]

#### Separating the stocks form NSE exchange and BSE exchange
Some of the stocks in the dataframe are repeated as the same stock sold on NSE and BSE are counted as 

The following method works to get a unique list of stocks

In [ ]:
r = [1]
for i in range(1,len(symbol)):
    l= symbol["Symbol"][i].split('.')
    r.append(l[0])


In [ ]:
dup = symbol["Symbol"]

In [ ]:
for i in range(len(r)):
    for j in range(i+1, len(r)):
        if r[i] == r[j]:
            dup = dup.drop(i)
            

In [ ]:
dup

In [ ]:
dup1 = dup.reset_index()

In [ ]:
len(dup)

In [ ]:
dup1

In [ ]:
del dup1["index"]

In [ ]:
dup1

Phewww!! finally I now have a dataframe with proper indexing and no repeating stocks. Now I need to save this as a csv file to be recalled later.

In [ ]:
dup1.to_csv("Pharma Stock Symbols from NSE and BSE.csv")

In [ ]:
list_of_symbols = []
for i in range(len(dup1)):
    list_of_symbols.append(dup1["Symbol"][i])

In [ ]:
list_of_symbols

In [ ]:
stock_data = yf.download(list_of_symbols, '2018-01-01', '2019-01-01')

In [ ]:
stock_data.to_csv("Pharma Stock - All Data.csv")

In [ ]:
close = stock_data["Adj Close"]

In [ ]:
type(close)

In [ ]:
close.describe()

In [ ]:
close.info()

In [ ]:
close.isna().sum().sum()

In [ ]:
del close['GLAND.NS']
del close['SUVENPHAR.NS']
del close['NOVARTIND.NS']

In [ ]:
close.isna().sum().sum()

In [ ]:
for cols in close.columns:
    if (close[cols].isna().sum() >0):
        del close[cols]

In [ ]:
close.isna().sum().sum()

saving the dataframe with adjusted close prices and all the NaN values droped.

In [ ]:
close.to_csv('Adj_Close.csv')

In [ ]:
returns = close.pct_change(1)

In [ ]:
returns = returns.dropna(axis = 0)

In [ ]:
returns

# Finding Pairs

## Finding pairs using the clustering method

In [ ]:
# select the columns that you are goin gto use for clustering :- volume and returns

metrics = ["Volume", "Adj Close"]

In [ ]:
stock_data1 = stock_data[metrics]

In [ ]:
stock_data1.isna().sum().sum()

In [ ]:
stock_data1 = stock_data1.dropna(axis=1)

In [ ]:
stock_data1.isna().sum().sum()

In [ ]:
returns =  stock_data1["Adj Close"].pct_change(1)

In [ ]:
returns =returns.dropna(axis=0)

In [ ]:
new_df = pd.concat([stock_data1, returns], axis = 1)

In [ ]:
new_df

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA().fit(returns)


In [ ]:
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components', fontsize = 15)
plt.ylabel('Variance (%)', fontsize = 15) 
plt.title('Explained Variance', fontsize = 20)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
results_dict = {}
num_of_clusters = 10


for k in range(2, num_of_clusters):
    print("-"*100)
    
    # define the next dictionary to hold all the results of this run.
    results_dict[k] = {}

    # create an instance of the model, and fit the training data to it.
    kmeans = KMeans(n_clusters=k, random_state=0).fit(returns)
    
    # define the silhouette score
    sil_score = metrics.silhouette_score(returns, kmeans.labels_, metric='euclidean')
    
    # store the different metrics
    results_dict[k]['silhouette_score'] = sil_score
    results_dict[k]['inertia'] = kmeans.inertia_
    results_dict[k]['score'] = kmeans.score
    results_dict[k]['model'] = kmeans
    
    # print the results    
    print("Number of Clusters: {}".format(k))
    print('Silhouette Score:', sil_score)

## Finding pairs using the double stage cointegration method

In [ ]:
returns.T

## Find pairs using simply the cointegration method